## Lab 5 Code!

We need to implement module that "reads files and stores their content in 2 string arrays of file names."

In [70]:
pos_file = "../datasets/movies_reviews/pos"
neg_file = "../datasets/movies_reviews/neg"

In [71]:
# So let's make a 2D array where we keep track of this. 

# We need to go through each file in "pos" and negative
import numpy as np
import sklearn
import pathlib as fl
import os as os

p = fl.Path(pos_file)
p.resolve
pos_fls = os.listdir(p)
    
words = []

# Go through pos:

for file_name in pos_fls:
    # Get all words in 'file_name' and store them in a string
    with open(pos_file + "/" + file_name) as f:
        words.append(f.read())
        
# Now store the break point for positives:
n = fl.Path(neg_file)
n.resolve
neg_fls = os.listdir(n)

neg_st = len(words)

# Add the negative files

for file_name in neg_fls:
    # Get all words in 'file_name' and store them in a string
    with open(neg_file + "/" + file_name) as f:
        words.append(f.read())
        
print(words[0:3])

['assume nothing . \nthe phrase is perhaps one of the most used of the 1990\'s , as first impressions and rumors are hardly ever what they seem to be . \nthe phrase especially goes for oscar novak , an architect who is the main focus of three to tango , a delightful , funny romantic comedy about assumptions and being yourself . \nnovak ( matthew perry ) , a shy , clumsy , chicago based architect , along with openly gay partner , peter steinberg ( oliver platt ) , fights for projects day in and day out . \none of these is the job of restoring a popular building for charles newman ( dylan mcdermott ) , a rich , well-known businessman . \ncharles immediately takes a liking to oscar , as he enjoys his personality and sense of humor . \nseeing oscar as someone he could trust , charles asks him to watch his girlfriend , an unpredictable , adventurous girl named amy post ( neve campbell ) , who makes a living by blowing glass . \ncharles wants to know who she talks to , what she does , and wh

# Vectorizer

We'll use the CountVectorizer form sklearn to convert our text documents into an array of word counts. I think it works by taking key words and reporting their total counts within that article. I believe it excludes non-key words (like articles, like "i

In [72]:
# So we now have an array with all the text from all the reviews (and its associated to a unique document)

from sklearn.feature_extraction.text import CountVectorizer


count_vectorizer = CountVectorizer()

vector = count_vectorizer.fit_transform(words)
arr = vector.toarray()

In [83]:
class_lbl = np.ones((neg_st,)) 
class_lbl = np.concatenate((class_lbl, np.zeros(((len(words) - neg_st),))))
class_lbl
re_class = class_lbl.reshape(-1)

In [84]:
# So now we have data (in arr), and classes (in re_class)
# Meaning we have an X and a Y

# Lets get a training and tesitng set:

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(arr, re_class, test_size=0.2)

# Naive Bayes

We're going to use the MultinomialNB, which purports to be good at dealing with word data (according to the docs). 

In [85]:
from sklearn.naive_bayes import MultinomialNB

naive = MultinomialNB()

naive.fit(X_train, Y_train)

MultinomialNB()

In [86]:
naive.score(X_test, Y_test)

0.8254364089775561

We get a score of 82.5% That's pretty good!

# Test on real life movie reviews

I've found some movie reviews for recent movies I've seen, so lets see how this goes. 
The order is neg-pos-neg-pos-pos

In [89]:
mine = "../datasets/movies_reviews/mine"
m = fl.Path(mine)
m.resolve
mine_fls = os.listdir(m)
    
true_class = [0, 1, 0, 1, 1]
mine_a = []

# Go through mine:

for file_name in mine_fls:
    # Get all words in 'file_name' and store them in a string
    if (file_name == ".DS_Store"):
        continue
    with open(mine + "/" + file_name) as f:
        mine_a.append(f.read())
        

# Now we need to vectorize mine_a
vec = count_vectorizer.transform(mine_a)
test_a = vec.toarray()

In [90]:
naive.predict(test_a) # Test our input

array([1., 1., 1., 1., 1.])

So we predict that all of them are positive...

This is not quite right, but it was right for 3/5 of them!

Here are the true classes:

true_class = [0, 1, 0, 1, 1] 

I think it likely predicts the negative reviews as positive since they didn't include many common negative words. The articles were written somewhat preteniously, so they likely included less common words which may have given our classifier a tougher time. 
